***Rigid Body Test***

Free fall test, and force measurement of rigid bodies with default simulation properties

In [1]:
from isaacsim import SimulationApp
simulation_app = SimulationApp({"headless": False})

# Force startup in paused mode
simulation_app.set_setting("/physics/autoStart", False)
simulation_app.set_setting("/app/runSim", False)


import numpy as np
import matplotlib.pyplot as plt
import csv, time, os
from datetime import datetime

from isaacsim.core.api import World
from isaacsim.core.api.objects import DynamicSphere, FixedCuboid
from isaacsim.core.prims import RigidPrim
from pxr import UsdGeom, UsdPhysics
import omni.ui as ui

# --- Deformable Cube for Compression Test ---
from isaacsim.core.prims import DeformablePrim

from isaacsim.core.api.materials.deformable_material import DeformableMaterial
from isaacsim.core.prims import SingleDeformablePrim
import isaacsim.core.utils.deformable_mesh_utils as deformableMeshUtils
from omni.physx.scripts import physicsUtils
import torch


print("Loaded Sim")

Starting kit application with the following args:  ['/home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/exts/isaacsim.simulation_app/isaacsim/simulation_app/simulation_app.py', '/home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/apps/isaacsim.exp.base.python.kit', '--/app/tokens/exe-path=/home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--/app/installSignalHandlers=0', '--ext-folder', '/home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/exts', '--ext-folder', '/home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/apps', '

2025-12-10T09:31:14Z [3,642ms] [Error] [asyncio] Exception in callback _patch_task.<locals>.step()
handle: <Handle _patch_task.<locals>.step()>
Traceback (most recent call last):
  File "/home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/kit/python/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x72812a9ff480> is already entered
2025-12-10T09:31:15Z [4,005ms] [Error] [asyncio] Task was destroyed but it is pending!
task: <Task pending name='Task-38' coro=<_async_in_context.<locals>.run_in_context() done, defined at /home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/kit/python/lib/python3.11/site-packages/ipykernel/utils.py:57> wait_for=<Task pending name='Task-39' coro=<Kernel.shell_main() running at /home/rjrosales/Simulation/Custom_IsaacSim/_build/linux-x86_64/release/kit/python/lib/python3.11/site-packages/ipykernel/kernel

Loaded Sim


### Simulation parameters

In [ ]:
DT = 1/60.0
SPHERE_RADIUS = 0.10
BLOCK_SIZE = 0.5
TEST_NAME = "sphere_compression"
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

CSV_FILE = f"{TEST_NAME}_{TIMESTAMP}.csv"
PLOT_FILE = f"{TEST_NAME}_{TIMESTAMP}.png"

running = False   # UI-controlled flag
disp_log = []
force_log = []


### GUI Panel

In [3]:
def create_world():
    world = World(
        stage_units_in_meters=1.0,
        backend="torch",     # REQUIRED for deformables
        device="cuda"        # uses GPU soft-body solver
    )
    stage = world.stage
    world.scene.add_default_ground_plane()
    return world, stage


def create_single_deformable_block(world, block_size=0.04, z_bottom=0.0):
    mesh_path = "/DeformableBlock"

    # 1) Make cube mesh (unit cube, triangulated)
    tri_points, tri_indices = deformableMeshUtils.createTriangleMeshCube(8)
    tri_points = np.array(tri_points) * block_size  # scale to e.g. 0.04 m

    mesh = UsdGeom.Mesh.Define(world.stage, mesh_path)
    mesh.GetPointsAttr().Set(tri_points.tolist())
    mesh.GetFaceVertexIndicesAttr().Set(tri_indices)
    mesh.GetFaceVertexCountsAttr().Set([3] * (len(tri_indices) // 3))

    # 2) Place bottom at z_bottom
    # The cube now spans [-block_size/2, +block_size/2] in local coords
    # So bottom is at -block_size/2 → we need +block_size/2 + z_bottom
    physicsUtils.setup_transform_as_scale_orient_translate(mesh)
    physicsUtils.set_or_add_translate_op(mesh, (0.0, 0.0, z_bottom + block_size/2))

    # 3) Material
    deform_mat = DeformableMaterial(
        prim_path="/World/DeformableMaterial",
        youngs_modulus=5e4,
        poissons_ratio=0.49,
        damping_scale=0.05,
        elasticity_damping=0.05,
        dynamic_friction=0.5,
    )

    # 4) Register SingleDeformablePrim
    deformable = SingleDeformablePrim(
        prim_path=mesh_path,
        name="silicone_block",
        deformable_material=deform_mat,
        simulation_hexahedral_resolution=3,
        solver_position_iteration_count=40,
        self_collision=True,
        collision_simplification=True,
    )

    world.scene.add(deformable)
    print("🟦 Single deformable block created at bottom z =", z_bottom)
    return deformable




def create_dynamic_sphere(world):
    return world.scene.add(
        DynamicSphere(
            prim_path="/Sphere",
            name="sphere",
            position=np.array([0, 0, 1.0]),
            radius=SPHERE_RADIUS,
            color=np.array([255, 255, 0]),
            mass=1.0
        )
    )


### World and Geometry Creation 

In [4]:
def create_prismatic_joint(stage):
    joint = UsdPhysics.PrismaticJoint.Define(stage, "/World/SphereJointZ")
    joint.CreateAxisAttr("Z")
    joint.CreateLowerLimitAttr(0.0)     # minimum sphere position
    joint.CreateUpperLimitAttr(1.5)     # max travel

    # Static reference frame
    xform = UsdGeom.Xform.Define(stage, "/World/RefFrame")
    joint.CreateBody0Rel().SetTargets(["/World/RefFrame"])

    # Sphere is the moving rigid body
    joint.CreateBody1Rel().SetTargets(["/Sphere"])

    # Position-based actuator
    drive = UsdPhysics.DriveAPI.Apply(joint.GetPrim(), "linear")
    drive.CreateTypeAttr().Set("position")
    drive.CreateStiffnessAttr().Set(40000)  # spring strength
    drive.CreateDampingAttr().Set(6000)     # reduce oscillations

    print("🔧 Prismatic joint attached to /Sphere")

    return drive


def create_contact_view(world):
    view = RigidPrim(
        prim_paths_expr="/Sphere",                  # the object being measured
        contact_filter_prim_paths_expr=["/DeformableBlock"],  # MUST match deformable mesh path
        name="sphere_contact_view",
        max_contact_count=128,                      # deformables create many contact nodes
    )

    world.scene.add(view)
    print("👀 Contact view added: Sphere <-> DeformableBlock")

    return view


In [1]:
def start_test():
    global running, disp_log, force_log
    running = True
    disp_log.clear()
    force_log.clear()
    print("▶ Test started.")


def stop_test():
    global running
    running = False
    print("⏹ Test stopped — data ready.")

### Simulation Loop

In [2]:
def run_test(world, drive, contact_view, sphere):
    time_log = []
    disp_log = []
    force_log = []

    target = 1.0
    direction = -1
    sim_time = 0.0   # simulation time

    print("⏸ Simulation waiting for PLAY...")

    while simulation_app.is_running():

        if world.is_playing():
            # ---- Time update ----
            sim_time += DT
            time_log.append(sim_time)

            # ---- Sphere displacement control ----
            target += 0.001 * direction
            if target <= 0.35:
                direction = +1
            elif target >= 1.00:
                direction = -1

            drive.GetTargetPositionAttr().Set(target)

            # Sphere world position is true displacement feedback
            z = sphere.get_world_pose()[0][2]
            disp_log.append(z)

            # ---- Contact force extraction ----
            result = contact_view.get_contact_force_data(dt=DT)

            if result:
                forces, _, normals, *_ = result

                if forces.shape[0] > 0:
                    # Torch-native operation → stays on GPU
                    fn = float(torch.sum(forces * normals))
                else:
                    fn = 0.0
            else:
                fn = 0.0

            force_log.append(fn)

        # Detect STOP pressed in Isaac Sim UI
        if world.is_stopped() and len(time_log) > 5:
            print("⏹ STOP detected — ending test")


### Data Logging

In [3]:
def save_csv(time, disp, force, tag="default"):
    """
    Save Force–Displacement–Time data to CSV using a naming tag.
    Example: tag="sphere_test" → sphere_test_data.csv
    """
    filename = f"{tag}_compression_data.csv"

    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["time_s", "displacement_m", "normal_force_N"])

        for t, d, fn in zip(time, disp, force):
            writer.writerow([t, d, fn])

    print(f"🧾 CSV saved → {filename}")
    return filename



def save_plot(time, disp, force):
    # ---- Plot 1: Force vs Displacement ----
    plt.figure(figsize=(7,4))
    plt.plot(disp, force, linewidth=2)
    plt.xlabel("Sphere Displacement (m)")
    plt.ylabel("Normal Force (N)")
    plt.title("Sphere Compression — Force vs Displacement")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(PLOT_FILE.replace(".png", "_disp.png"), dpi=300)
    print("📈 Plot saved →", PLOT_FILE.replace(".png", "_disp.png"))
    plt.show()

    # ---- Plot 2: Force vs Time ----
    plt.figure(figsize=(7,4))
    plt.plot(time, force, linewidth=2)
    plt.xlabel("Time (s)")
    plt.ylabel("Normal Force (N)")
    plt.title("Sphere Compression — Force vs Time")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(PLOT_FILE.replace(".png", "_time.png"), dpi=300)
    print("⏱️ Plot saved →", PLOT_FILE.replace(".png", "_time.png"))
    plt.show()


In [4]:
world, stage = create_world()
create_deformable_block(world)
sphere = create_dynamic_sphere(world)
drive = create_prismatic_joint(stage)
contact_view = create_contact_view(world)
world.reset()
print("Deformable count:", DeformablePrim(prim_paths_expr="/*").count)



NameError: name 'create_world' is not defined

In [ ]:

time, disp, force = run_test(world, drive, contact_view,sphere)
# save_csv(time, disp, force)
save_plot(time, disp, force)


In [3]:
simulation_app.close()

NameError: name 'disp' is not defined